In [ ]:
import pandas as pd
import re
from g2pk import G2p

# G2P 객체 생성
g2p = G2p()

# 특수 토큰과 대체 단어 설정
special_tokens = {
    '#Person1#': '임시토큰11',
    '#Person2#': '임시토큰12',
    '#Person3#': '임시토큰13',
    '#PhoneNumber#': '임시토큰14',
    '#Address#': '임시토큰15',
    '#PassportNumber#': '임시토큰16'
}

# 특수 토큰을 임시로 대체하는 함수
def replace_special_tokens(text):
    for token, replacement in special_tokens.items():
        text = text.replace(token, replacement)
    return text

# 특수 토큰을 원래 상태로 복원하는 함수
def restore_special_tokens(text):
    for replacement, token in special_tokens.items():
        text = text.replace(token, replacement)
    return text

# summary와 dialogue에서 고유한 영어 단어 추출 함수
def extract_unique_english_words(text):
    words = text.split()
    english_words = set()
    
    for word in words:
        cleaned_word = re.sub(r'[^a-zA-Z]', '', word)
        if cleaned_word and cleaned_word not in special_tokens.values():
            english_words.add(cleaned_word)
    
    return list(english_words)

# 영어 이름을 한국어 발음으로 변환하는 함수
def english_to_korean_pronunciation(name):
    return g2p(name)

# Summary와 dialogue에서 영어 이름을 한국어 발음으로 교체하는 함수
def replace_english_names(row):
    dialogue = row['dialogue']
    summary = row['summary']
    
    # 특수 토큰을 임시로 대체
    dialogue = replace_special_tokens(dialogue)
    summary = replace_special_tokens(summary)
    
    # 1. summary와 dialogue에서 고유한 영어 단어 추출
    english_words_summary = extract_unique_english_words(summary)
    english_words_dialogue = extract_unique_english_words(dialogue)
    
    # 2. summary에만 있는 영어 단어 처리
    unique_to_summary = list(set(english_words_summary) - set(english_words_dialogue))
    
    # 3. summary와 dialogue에 모두 등장하는 영어 단어 처리
    common_english_words = list(set(english_words_summary) & set(english_words_dialogue))
    
    # 4. summary에만 있는 영어 단어 번역 및 교체
    if unique_to_summary:
        for e_word in unique_to_summary:
            k_word = english_to_korean_pronunciation(e_word)
            summary = summary.replace(e_word, k_word)
            print(f'Replacing {e_word} with {k_word} in summary only.')
    
    # 5. summary와 dialogue에 모두 있는 영어 단어 번역 및 교체
    if common_english_words:
        for e_word in common_english_words:
            k_word = english_to_korean_pronunciation(e_word)
            summary = summary.replace(e_word, k_word)
            dialogue = dialogue.replace(e_word, k_word)
            print(f'Replacing {e_word} with {k_word} in both summary and dialogue.')
    
    # 업데이트된 값을 반환
    row['summary'] = summary
    row['dialogue'] = dialogue
    
    return row

# 데이터프레임 생성 예시
# new_ordered_df = pd.DataFrame({
#     'dialogue': ordered_df[ordered_df['fname'] == 'train_3104']['dialogue'],
#     'summary': ordered_df[ordered_df['fname'] == 'train_3104']['summary']
# })

# Summary와 dialogue에서 영어 이름을 한국어 발음으로 교체
new_ordered_df = new_ordered_df.apply(replace_english_names, axis=1)

# 특수 토큰을 원래 상태로 복원
def restore_tokens(row):
    row['dialogue'] = restore_special_tokens(row['dialogue'])
    row['summary'] = restore_special_tokens(row['summary'])
    return row

# 복원 작업 적용
new_ordered_df = new_ordered_df.apply(restore_tokens, axis=1)

# # 결과 확인
# print(new_ordered_df['summary'])
# print(new_ordered_df['dialogue'])

In [ ]:
import re
from transformers import AutoTokenizer, AutoModelForTokenClassification, pipeline
from g2pk import G2p
import pandas as pd
from googletrans import Translator

# G2P 객체 생성
g2p = G2p()
translator =Translator()

# NER 모델 로드
tokenizer_en = AutoTokenizer.from_pretrained("dbmdz/bert-large-cased-finetuned-conll03-english")
model_en = AutoModelForTokenClassification.from_pretrained("dbmdz/bert-large-cased-finetuned-conll03-english")
ner_en = pipeline("ner", model=model_en, tokenizer=tokenizer_en)



special_tokens = {
    '#Address#': '#임시주소153#',
    '#CarNumber#': '#임시차량번호154#',
    '#CardNumber#': '#임시카드번호155#',
    '#DateOfBirth#': '#임시생년월일156#',
    '#Email#': '#임시이메일157#',
    '#PassportNumber#': '#임시여권번호158#',
    '#Person#': '#임시인물159#',
    '#Person1#': '#임시인물160#',
    '#Person2#': '#임시인물161#',
    '#Person3#': '#임시인물162#',
    '#Person4#': '#임시인물163#',
    '#Person5#': '#임시인물164#',
    '#Person6#': '#임시인물165#',
    '#Person7#': '#임시인물166#',
    '#PhoneNumber#': '#임시전화번호167#',
    '#SSN#': '#임시주민번호168#'
}
# 특수 토큰을 임시로 대체하는 함수
def replace_special_tokens(text):
    for token, replacement in special_tokens.items():
        text = text.replace(token, replacement)
    return text

# 특수 토큰을 원래 상태로 복원하는 함수
def restore_special_tokens(text):
    for replacement, token in special_tokens.items():
        text = text.replace(token, replacement)
    return text

def detect_language(word):
    is_korean = all('가' <= char <= '힣' for char in word)

    if is_korean:
        return True
    else:
        return False

# 한글 자모 분리를 위한 유니코드 조작 함수
def decompose_hangul(syllable):
    base_code = ord(syllable) - 0xAC00
    choseong = base_code // 588
    jungseong = (base_code - (choseong * 588)) // 28
    jongseong = base_code % 28
    choseong_base = 0x1100
    jungseong_base = 0x1161
    jongseong_base = 0x11A7

    if jongseong != 0:
        return chr(choseong_base + choseong), chr(jungseong_base + jungseong), chr(jongseong_base + jongseong)
    else:
        return chr(choseong_base + choseong), chr(jungseong_base + jungseong), None  # 종성이 없으면 None

def split_syllables(text):
    result = []
    for char in text:
        if '가' <= char <= '힣':  # 한글 음절 여부 확인
            result.append(decompose_hangul(char))
        else:
            result.append((char,))
    return result

def normalize_consonant(consonant):
    if consonant is None:
        return None
    consonant_code = ord(consonant)
    
    # 초성을 종성으로 변환
    if 0x1100 <= consonant_code <= 0x1112:  # 초성 범위
        return chr(consonant_code + 169)  # 초성을 종성으로 변환
    
    # 종성을 초성으로 변환
    elif 0x11A8 <= consonant_code <= 0x11C2:  # 종성 범위
        return chr(consonant_code - 169)  # 종성을 초성으로 변환
    
    return consonant

# 음절 단위로 유사도 계산 및 초성/종성 변환 후 비교
def calculate_similarity(word1, word2):
    word1_split = split_syllables(word1)
    word2_split = split_syllables(word2)

    match_count = 0
    max_count = max(len(word1_split), len(word2_split))

    i = 0
    while i < min(len(word1_split), len(word2_split)):
        syllable1 = word1_split[i]
        syllable2 = word2_split[i]

        print(f"Comparing syllable '{syllable1}' with '{syllable2}'")

        # 초성, 중성 비교
       
        if syllable1[0] == syllable2[0]:
            match_count += 4/11  # 각 성분이 일치할 때마다 1/3점 부여
            print(f"Partial match for '{syllable1[0]}' and '{syllable2[0]}'. Match count: {match_count}")
        else:
            print(f"No match for '{syllable1[0]}' and '{syllable2[0]}'")
               
        if syllable1[1] == syllable2[1]:
            match_count += 1/3  # 각 성분이 일치할 때마다 1/3점 부여
            print(f"Partial match for '{syllable1[1]}' and '{syllable2[1]}'. Match count: {match_count}")
        else:
            print(f"No match for '{syllable1[1]}' and '{syllable2[1]}'")

        # 종성 비교
        if syllable1[2] == syllable2[2]:  # 종성이 일치할 때
            match_count += 1/3
            print(f"Match for 종성 '{syllable1[2]}' and '{syllable2[2]}'. Match count: {match_count}")
        elif syllable1[2] is None and syllable2[2] is not None:  # 첫 단어에 종성이 없고, 두 번째 단어에 종성이 있는 경우
            if i + 1 < len(word1_split):
                next_choseong = normalize_consonant(word1_split[i + 1][0])
                if next_choseong == syllable2[2]:
                    match_count += 1/3
                    print(f"Match for 종성 '{syllable2[2]}' with next 초성 '{next_choseong}'. Match count: {match_count}")
                else:
                    print(f"No match for 종성 '{syllable1[2]}' and '{syllable2[2]}'")
        elif syllable2[2] is None and syllable1[2] is not None:  # 첫 단어에 종성이 있고, 두 번째 단어에 종성이 없는 경우
            if i + 1 < len(word2_split):
                next_choseong = normalize_consonant(word2_split[i + 1][0])
                if next_choseong == syllable1[2]:
                    match_count += 1/3
                    print(f"Match for 종성 '{syllable1[2]}' with next 초성 '{next_choseong}'. Match count: {match_count}")
                else:
                    print(f"No match for 종성 '{syllable1[2]}' and '{syllable2[2]}'")
        else:
            print(f"No match for 종성 '{syllable1[2]}' and '{syllable2[2]}'")

        i += 1

    # 유사도 비율 계산
    similarity = match_count / max_count
    print(f"Final similarity: {similarity:.2f}\n")

    return similarity

# 영어 이름을 한국어 발음으로 변환하고, 동일한 이름이 양쪽에 있으면 그대로 변환
def convert_word(word, entity_type, context_words, is_common):
    if entity_type in ['B-PER', 'I-PER']:  # 인물 이름인 경우 처리
        korean_pronunciation = g2p(word)
        google_translation = translator.translate(word, src='en', dest='ko').text
        print(f"Korean Pronunciation of '{word}': {korean_pronunciation}")
        print(is_common)
        print(f"Korean Pronunciation of '{word}': {google_translation} from google")
        
        
        if is_common:  # 동일한 이름이 양쪽에 있을 경우
            if detect_language(google_translation): # 구글 번역이 한글일 경우
                return google_translation
            elif detect_language(korean_pronunciation): # G2P 가 한글일 경우
                return korean_pronunciation
            else:
                return google_translation
        elif context_words:  # Context words가 존재하는지 확인
            if detect_language(google_translation) and detect_language(korean_pronunciation):
                
                best_match = max(context_words, key=lambda w: calculate_similarity(korean_pronunciation, w), default=korean_pronunciation)
                best_match_google = max(context_words, key=lambda w: calculate_similarity(google_translation, w), default=google_translation)
                similarity_score = calculate_similarity(korean_pronunciation, best_match)
                similarity_score_google = calculate_similarity(google_translation, best_match_google)
                print(f"Best match for '{korean_pronunciation}' in context: '{best_match}' with similarity {similarity_score:.2f}")
                print(f"Best match for '{google_translation}' in context: '{best_match_google}' with similarity {similarity_score_google:.2f}")
                
                if similarity_score <= similarity_score_google:
                    best_match = best_match_google
                    similarity_score = similarity_score_google
                
                if similarity_score > 0.6:  # 유사도 임계값을 낮춰서 매칭 범위를 확장
                    return best_match
                else:
                    return google_translation
            elif detect_language(google_translation):   
                                
                best_match_google = max(context_words, key=lambda w: calculate_similarity(google_translation, w), default=google_translation)
                similarity_score_google = calculate_similarity(google_translation, best_match_google)
                print(f"Best match for '{google_translation}' in context: '{best_match_google}' with similarity {similarity_score_google:.2f}")
                

                best_match = best_match_google
                similarity_score = similarity_score_google
                if similarity_score > 0.6:  # 유사도 임계값을 낮춰서 매칭 범위를 확장
                    return best_match
                else:
                    return google_translation
                
            elif detect_language(korean_pronunciation):
                
                best_match = max(context_words, key=lambda w: calculate_similarity(korean_pronunciation, w), default=korean_pronunciation)
                similarity_score = calculate_similarity(korean_pronunciation, best_match)
                print(f"Best match for '{korean_pronunciation}' in context: '{best_match}' with similarity {similarity_score:.2f}")
                
                if similarity_score > 0.6:  # 유사도 임계값을 낮춰서 매칭 범위를 확장
                    return best_match
                else:
                    return korean_pronunciation
            else:
                return word
                
        elif detect_language(google_translation):
            return google_translation
        elif detect_language(korean_pronunciation):
            return korean_pronunciation
        else:
            return word
    else:
        return word  # 기타 개체는 그대로 유지

# 엔티티 인식 후 변환하는 메인 함수
def process_text(row):
    dialogue = row['dialogue']
    summary = row['summary']
    
    english_words_dialogue = extract_english_words(dialogue)
    print(english_words_dialogue)
    english_words_summary = extract_english_words(summary)
    print(english_words_summary)
    english_words = list(set(english_words_dialogue + english_words_summary))
    entities = classify_entities(english_words, ner_en)  # 영어 NER 사용
    
    context_words = re.findall(r'\b[가-힣]+\b', dialogue + " " + summary)  # 한국어 단어 추출
    print(context_words)

    for word, entity_type in entities.items():
        is_common = word in english_words_dialogue and word in english_words_summary
        converted_word = convert_word(word, entity_type, context_words, is_common)
        dialogue = dialogue.replace(word, converted_word)
        summary = summary.replace(word, converted_word)
    
    row['new_dialogue'] = dialogue
    row['new_summary'] = summary
    return row

# 영어 단어 추출 함수
def extract_english_words(text):
    # 정규 표현식을 사용하여 영어 단어만 추출
    english_words = re.findall(r'\b[A-Za-z]+\b', text)
    return english_words


# 엔티티 분류 함수
def classify_entities(words, ner_pipeline):
    entities = {}
    for word in words:
        ner_results = ner_pipeline(word)
        if ner_results:
            entities[word] = ner_results[0]['entity']
    print(f"Extracted Entities: {entities}")  # NER 모델 출력 확인
    return entities




df_filtered['dialogue'] = df_filtered['dialogue'].apply(replace_special_tokens)
df_filtered['summary'] = df_filtered['summary'].apply(replace_special_tokens)

df_filtered = df_filtered.apply(process_text, axis=1)

# 특수 토큰 복원
df_filtered['dialogue'] = df_filtered['dialogue'].apply(restore_special_tokens)
df_filtered['summary'] = df_filtered['summary'].apply(restore_special_tokens)

df_filtered['new_dialogue'] = df_filtered['new_dialogue'].apply(restore_special_tokens)
df_filtered['new_summary'] = df_filtered['new_summary'].apply(restore_special_tokens)



In [ ]:
import pandas as pd
from googletrans import Translator

# 번역기 객체 생성
translator = Translator()

# 예제 DataFrame (실제 df로 대체하세요)


# 1. english_words_count가 0이 아닌 행 필터링
df_filtered = new_ordered_df[new_ordered_df['english_words_count'] > 0].copy()

# 2. dialogue와 summary를 영어로 번역한 뒤 다시 한국어로 재번역
df_filtered['dialogue_translated'] = df_filtered['dialogue'].apply(lambda x: translator.translate(translator.translate(x, src='ko', dest='en').text, src='en', dest='ko').text)
df_filtered['summary_translated'] = df_filtered['summary'].apply(lambda x: translator.translate(translator.translate(x, src='ko', dest='en').text, src='en', dest='ko').text)

# 3. 원래 df에 재번역된 값 채워넣기
new_ordered_df.loc[new_ordered_df['english_words_count'] > 0, 'dialogue'] = df_filtered['dialogue_translated']
new_ordered_df.loc[new_ordered_df['english_words_count'] > 0, 'summary'] = df_filtered['summary_translated']

# 결과 확인
print(new_ordered_df[['dialogue', 'summary']])

In [ ]:
new_ordered_df = pd.read_csv('/dj/new_filtered.csv')


In [ ]:
df = pd.DataFrame({
    'dialogue': [
        "#Person1#: 안녕, Peter Johnson! 어떻게 지냈어? 우리가 마지막으로 만난 지 꽤 됐네. "
        "#Person2#: 안녕, 죤 스미트! 난 잘 지냈어. 어제 뉴욕에서 돌아왔어. "
        "#Person1#: 뉴욕에서 좋은 시간 보냈어? 거기서 뭘 했어? "
        "#Person2#: 네, 정말 좋은 시간이었어. 주로 친구들을 만나고 몇 가지 비즈니스 미팅도 했어. "
        "#Person1#: 좋았겠네! 나도 뉴욕에 가본 지 오래됐어. 다음에 같이 가자. "
        "#Person2#: 좋은 생각이야. 그때는 우리 둘 다 여유를 좀 가질 수 있었으면 좋겠어. "
    ],
    'summary': [
        "#Person7# Peter Johnson and John Smith catch up after not seeing each other for a while. "
        "John mentions that he recently returned from New York, where he spent time meeting friends and attending business meetings. "
        "They both agree to plan a trip to New York together in the future."
    ]
})


import pandas as pd
import re


# 특수 토큰 설정 및 대체 단어 정의
special_tokens = {
    '#Address#': '#임시주소153#',
    '#CarNumber#': '#임시차량번호154#',
    '#CardNumber#': '#임시카드번호155#',
    '#DateOfBirth#': '#임시생년월일156#',
    '#Email#': '#임시이메일157#',
    '#PassportNumber#': '#임시여권번호158#',
    '#Person#': '#임시인물159#',
    '#Person1#': '#임시인물160#',
    '#Person2#': '#임시인물161#',
    '#Person3#': '#임시인물162#',
    '#Person4#': '#임시인물163#',
    '#Person5#': '#임시인물164#',
    '#Person6#': '#임시인물165#',
    '#Person7#': '#임시인물166#',
    '#PhoneNumber#': '#임시전화번호167#',
    '#SSN#': '#임시주민번호168#'
}
# 특수 토큰을 임시로 대체하는 함수
def replace_special_tokens(text):
    for token, replacement in special_tokens.items():
        text = text.replace(token, replacement)
    return text

# 특수 토큰을 원래 상태로 복원하는 함수
def restore_special_tokens(text):
    for replacement, token in special_tokens.items():
        text = text.replace(token, replacement)
    return text



df['dialogue'] = df['dialogue'].apply(replace_special_tokens)
df['summary'] = df['summary'].apply(replace_special_tokens)


# 특수 토큰 복원
df['dialogue'] = df['dialogue'].apply(restore_special_tokens)
df['summary'] = df['summary'].apply(restore_special_tokens)